In [18]:
#性能测量
def meansure_performance(y_predict_lis):
    TP=0
    FP=0
    TN=0
    FN=0
    #测试样本数
    sample_num=x_test.shape[0]
    for i in range(sample_num):
        y_predict=y_predict_lis[i]
        if y_predict==1 and y_test[i]==1:
            TP+=1
        elif y_predict==-1 and y_test[i]==-1:
            TN+=1
        elif y_predict==1 and y_test[i]==-1:
    #false poistive
            FP+=1
        elif y_predict==-1 and y_test[i]==1:
    #false negative
            FN+=1 
    accuracy=(TP+TN)/(TP+FP+TN+FN)
    recall=TP/(TP+FN)
    specificity=TN/(TN+FP)
    precision=TP/(TP+FP)
    F_measuer=(2*precision*recall)/(precision+recall)
    return accuracy,recall,specificity,precision,F_measuer

In [19]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            #print(i)
            i-=1
            

# 白云

In [21]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_baiyunshan.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
baiyun_accuracy=[i for i in range(10)]
baiyun_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [22]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    baiyun_accuracy[t]=performan_rfe_reliefF_SVM[0]
    baiyun_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


# 贵州

In [23]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_guizhou.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
guizhou_accuracy=[i for i in range(10)]
guizhou_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [24]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    guizhou_accuracy[t]=performan_rfe_reliefF_SVM[0]
    guizhou_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


In [39]:
guizhou_accuracy

[0.7413167444271643,
 0.7622903412377097,
 0.7547416612164813,
 0.7501881113619263,
 0.7573073516386183,
 0.7588805166846071,
 0.7434842249657064,
 0.7296786389413988,
 0.7416413373860182,
 0.7441860465116279]

# jinhui

In [25]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinhuijiu.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
jinhui_accuracy=[i for i in range(10)]
jinhui_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [26]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    jinhui_accuracy[t]=performan_rfe_reliefF_SVM[0]
    jinhui_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


In [40]:
jinhui_accuracy

[0.6111975116640747,
 0.7090803932909197,
 0.724002616088947,
 0.764484574868322,
 0.8104517271922055,
 0.8180839612486545,
 0.8148148148148148,
 0.8185255198487713,
 0.8085106382978723,
 0.875968992248062]

# jinshi

In [27]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinshi.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
jinshi_accuracy=[i for i in range(10)]
jinshi_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [28]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    jinshi_accuracy[t]=performan_rfe_reliefF_SVM[0]
    jinshi_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


In [41]:
jinshi_accuracy

[0.5090720580611716,
 0.7628687102371313,
 0.7920209287115761,
 0.8126410835214447,
 0.804251550044287,
 0.8083961248654468,
 0.803840877914952,
 0.7977315689981096,
 0.8237082066869301,
 0.8449612403100775]

# laobai

In [29]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_laobai.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
laobai_accuracy=[i for i in range(10)]
laobai_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [30]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    laobai_accuracy[t]=performan_rfe_reliefF_SVM[0]
    laobai_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


In [42]:
laobai_accuracy

[0.45049248315189216,
 0.626951995373048,
 0.7338129496402878,
 0.746425884123401,
 0.7936226749335695,
 0.8159311087190527,
 0.8285322359396433,
 0.7958412098298677,
 0.7993920972644377,
 0.8294573643410853]

# shanxi

In [31]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_shanxi.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
shanxi_accuracy=[i for i in range(10)]
shanxi_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [32]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    shanxi_accuracy[t]=performan_rfe_reliefF_SVM[0]
    shanxi_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


# tongren

In [33]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_tongren.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
tongren_accuracy=[i for i in range(10)]
tongren_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [34]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    tongren_accuracy[t]=performan_rfe_reliefF_SVM[0]
    tongren_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


# weiwei

In [35]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_weiwei.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
weiwei_accuracy=[i for i in range(10)]
weiwei_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [36]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    weiwei_accuracy[t]=performan_rfe_reliefF_SVM[0]
    weiwei_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


# yanghe

In [37]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_yanghe.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
yanghe_accuracy=[i for i in range(10)]
yanghe_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [38]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(C=0.1, kernel='rbf',gamma=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    yanghe_accuracy[t]=performan_rfe_reliefF_SVM[0]
    yanghe_F_measure[t]=performan_rfe_reliefF_SVM[4]
    print(t)
    t=t+1

0
1
2
3
4
5
6
7
8
9


In [45]:
Gassian_kernel_svm_accuracy=[]
Gassian_kernel_svm_accuracy.append(baiyun_accuracy)
Gassian_kernel_svm_accuracy.append(guizhou_accuracy)
Gassian_kernel_svm_accuracy.append(jinhui_accuracy)
Gassian_kernel_svm_accuracy.append(jinshi_accuracy)
Gassian_kernel_svm_accuracy.append(laobai_accuracy)
Gassian_kernel_svm_accuracy.append(shanxi_accuracy)
Gassian_kernel_svm_accuracy.append(tongren_accuracy)
Gassian_kernel_svm_accuracy.append(weiwei_accuracy)
Gassian_kernel_svm_accuracy.append(yanghe_accuracy)

In [54]:
colu=[200,400,600,800,1000,1200,1400,1600,1800,2000]
Gassian_kernel_svm_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(Gassian_kernel_svm_accuracy))
Gassian_kernel_svm_acurracy_dataframe.to_csv("Gaussian_kernel_svm_accuracy.csv")

In [51]:
Gassian_kernel_svm_F_measure=[]
Gassian_kernel_svm_F_measure.append(baiyun_F_measure)
Gassian_kernel_svm_F_measure.append(guizhou_F_measure)
Gassian_kernel_svm_F_measure.append(jinhui_F_measure)
Gassian_kernel_svm_F_measure.append(jinshi_F_measure)
Gassian_kernel_svm_F_measure.append(laobai_F_measure)
Gassian_kernel_svm_F_measure.append(shanxi_F_measure)
Gassian_kernel_svm_F_measure.append(tongren_F_measure)
Gassian_kernel_svm_F_measure.append(weiwei_F_measure)
Gassian_kernel_svm_F_measure.append(yanghe_F_measure)

In [55]:
colu=[200,400,600,800,1000,1200,1400,1600,1800,2000]
Gassian_kernel_svm_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(Gassian_kernel_svm_F_measure))
Gassian_kernel_svm_F_measure_dataframe.to_csv("Gaussian_kernel_svm_F_measure.csv")